# Tutorial: Electrothermal Component Telemetry

**Audience:** users already comfortable with Pulsim YAML + transient execution.

**You will learn:**

- how to run an electrothermal benchmark netlist,
- how to read `SimulationResult.component_electrothermal`,
- how to check consistency against `loss_summary` and `thermal_summary`.


## Outline

1. Locate repo root and import `pulsim`.
2. Load and run `buck_electrothermal.yaml`.
3. Inspect per-component electrothermal telemetry.
4. Cross-check aggregate consistency (`loss_summary`/`thermal_summary`).


In [ ]:
# Setup cell: keep it short and deterministic
from pathlib import Path
import math


def find_repo_root(start: Path) -> Path:
    here = start.resolve()
    for candidate in (here, *here.parents):
        if (candidate / "pyproject.toml").is_file() and (candidate / "benchmarks").is_dir():
            return candidate
    raise RuntimeError("Could not locate repository root from current working directory")

repo_root = find_repo_root(Path.cwd())

try:
    import pulsim as ps
except Exception as exc:
    raise RuntimeError(
        "Failed to import pulsim. Run notebook with PYTHONPATH=build/python."
    ) from exc

repo_root


## Step 1 - Run an electrothermal case

This uses the same netlist that backs the electrothermal benchmark gate.

Expected result:

- `result.success == True`
- deterministic entries in `result.component_electrothermal` for all non-virtual components


In [ ]:
# Minimal working example
netlist_path = repo_root / "benchmarks" / "circuits" / "buck_electrothermal.yaml"

parser_opts = ps.YamlParserOptions()
parser_opts.strict = False
parser = ps.YamlParser(parser_opts)
circuit, options = parser.load(str(netlist_path))
if parser.errors:
    raise RuntimeError("YAML parser errors:\\n" + "\\n".join(parser.errors))

options.newton_options.num_nodes = int(circuit.num_nodes())
options.newton_options.num_branches = int(circuit.num_branches())

sim = ps.Simulator(circuit, options)
result = sim.run_transient(circuit.initial_state())

print("success:", result.success)
print("message:", result.message)
print("components reported:", len(result.component_electrothermal))
print("components declared:", circuit.num_devices())

rows = sorted(result.component_electrothermal, key=lambda item: item.component_name)
print()
print("Per-component (name, thermal_enabled, total_loss, total_energy, peak_temperature)")
for item in rows:
    print(
        f"- {item.component_name:10s} thermal={str(item.thermal_enabled):5s} "
        f"P={item.total_loss:.6e} E={item.total_energy:.6e} Tpk={item.peak_temperature:.4f}"
    )

component_total_loss = sum(item.total_loss for item in rows)
summary_total_loss = float(result.loss_summary.total_loss)
loss_denom = max(abs(component_total_loss), abs(summary_total_loss), 1e-12)
loss_rel_err = abs(component_total_loss - summary_total_loss) / loss_denom

component_peak_temp = max(item.peak_temperature for item in rows) if rows else result.thermal_summary.ambient
summary_peak_temp = float(result.thermal_summary.max_temperature)
temp_denom = max(abs(component_peak_temp), abs(summary_peak_temp), 1e-12)
temp_rel_err = abs(component_peak_temp - summary_peak_temp) / temp_denom

print()
print("Consistency checks")
print("- loss rel error:", f"{loss_rel_err:.3e}")
print("- thermal peak rel error:", f"{temp_rel_err:.3e}")
print("- thermal summary enabled:", result.thermal_summary.enabled)

assert result.success
assert len(rows) == int(circuit.num_devices())
assert math.isfinite(loss_rel_err)
assert math.isfinite(temp_rel_err)


## Exercises

1. Change `simulation.thermal.policy` in the YAML to `loss_only` and compare temperatures.
2. Increase `M1.thermal.rth` by 2x and observe `peak_temperature` delta.
3. Add one extra passive component and verify it still appears in `component_electrothermal`.


In [ ]:
# Exercise answer scaffold
# 1) Duplicate netlist to a temporary file and edit thermal values.
# 2) Re-run parser + simulator.
# 3) Compare rows from result.component_electrothermal.

# TODO: implement your experiment here.
